In [1]:
import pandas as pd
import numpy as np
import random
import re,requests,time
import urllib
import lxml
from tqdm import tqdm
import json
from urllib import error
from bs4 import BeautifulSoup
import os
from datetime import datetime
import datetime
from lxml import etree
import math
import ssl

In [2]:
uuids=pd.read_excel(r"轻松筹uuid.xlsx",header=None)

In [3]:
uuids

,0
0,a9063506-2ad3-4ec6-9d97-02bb99acdf6b
1,9d0110f1-a870-4341-8923-0189f7782efa
2,d4895730-a695-4e75-a2e0-b47be9d84c61
3,8332e46a-faab-4477-b5b1-a7abc9ba13ac
4,028da1b4-c77c-4aa2-813d-bb9735cf3567
...,...
179,6e9110cc-3706-450a-89ce-3471411a1096
180,851af755-cf78-4e8e-b0e9-d3f92db5b379
181,1dab6826-8bb3-4f62-842e-065b4341ee64
182,9c9ced40-8590-4ca3-a04d-8b611bce4d21


In [4]:
uuid_cut=uuids.loc[:10,0]
uuid_cut

0     a9063506-2ad3-4ec6-9d97-02bb99acdf6b
1     9d0110f1-a870-4341-8923-0189f7782efa
2     d4895730-a695-4e75-a2e0-b47be9d84c61
3     8332e46a-faab-4477-b5b1-a7abc9ba13ac
4     028da1b4-c77c-4aa2-813d-bb9735cf3567
5     187fd170-3260-48e9-98dc-5f27fe78a0eb
6     3c5cce23-5070-47e3-bc1c-012aa1c26ea6
7     904b4d8e-bed2-423c-856a-e09e64ca30d3
8     fa21c0b8-a33b-4b97-bccc-25d9e62b7603
9     5b19630f-a4b6-4c0a-827f-7c0d75971f62
10    4d65f94f-4c69-4f04-9799-fc7f2959e4d7
Name: 0, dtype: object

In [40]:
# uuid_cut=uuids.loc[32:50,0].tolist()
# uuid_cut[0]

'df34a086-0bf9-40b9-bd29-600d8ace7115'

In [8]:
# 项目基本信息：年纪、地区
def get_baseinfo(uuid_cut):
    url = 'https://gateway.qschou.com/v3.0.0/project/index/auth3/{}'.format(uuid_cut)
    headers = {'Host': 'gateway.qschou.com', 'Connection': 'keep-alive', 'Content-Length': '9357', 'cookie': '', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 MicroMessenger/7.0.20.1781(0x6700143B) NetType/WIFI MiniProgramEnv/Windows WindowsWechat/WMPF XWEB/6939', 'charset': 'utf-8', 'Accept-Encoding': 'gzip,compress,br,deflate', 'Content-Type': 'application/json; charset=UTF-8','Referer': 'referer: https://servicewechat.com/wx41b2d3a971bf407b/121/page-frame.html'}
    response = requests.get(url)
    data = response.json()
    labels=data['data']['publisher_relation']
    if labels['patient_province'] is not None: 
    #  病人所处地区
        province=labels['patient_province']
    if labels['patient_age'] is not None: 
    #     病人年纪
        age=labels['patient_age']
    return province,age

In [9]:
# 项目进展披露详尽性
def get_jz(uuid_cut):
#     for i in range(len(uuid_cut)):
    url = 'https://gateway.qschou.com/v3.0.0/feed/project?uuid={}'.format(uuid_cut)
    headers = {'Host': 'gateway.qschou.com', 'Connection': 'keep-alive', 'Content-Length': '9357', 'cookie': '', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 MicroMessenger/7.0.20.1781(0x6700143B) NetType/WIFI MiniProgramEnv/Windows WindowsWechat/WMPF XWEB/6939', 'charset': 'utf-8', 'Accept-Encoding': 'gzip,compress,br,deflate', 'Content-Type': 'application/json; charset=UTF-8','Referer': 'referer: https://servicewechat.com/wx41b2d3a971bf407b/121/page-frame.html'}
    response = requests.get(url)
    data = response.json()
    image_count = 0
    content_count=0
    for topic in data["data"]:
    #       更新图片数量
        if topic["images"] is not None:
            image_count += len(topic["images"])
    #       更新字数
        if topic["content"] is not None:
            for content in topic["content"]:
                for item in content:
                     content_count += len(item["text"])
    #   进展更新次数
    up_times=len(data["data"])
    return up_times,image_count,content_count

In [10]:
# 亲友认证
def get_qy(uuid_cut):
#     for i in range(len(uuid_cut)):
    url = 'https://gateway.qschou.com/v3.0.0/project/index/newverify/{}'.format(uuid_cut)
    headers = {'Host': 'gateway.qschou.com', 'Connection': 'keep-alive', 'Content-Length': '9357', 'cookie': '', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 MicroMessenger/7.0.20.1781(0x6700143B) NetType/WIFI MiniProgramEnv/Windows WindowsWechat/WMPF XWEB/6939', 'charset': 'utf-8', 'Accept-Encoding': 'gzip,compress,br,deflate', 'Content-Type': 'application/json; charset=UTF-8','Referer': 'referer: https://servicewechat.com/wx41b2d3a971bf407b/121/page-frame.html'}
    response = requests.get(url)
    data = response.json()
#     亲友认证
    prove_total=data['data']['prove_total']
    return prove_total

In [11]:
# 项目评论信息
def get_pl(backer_count,uuid_cut):
    allpls=[]
    allhdpls=[]
    allzs=[]
# 当前页面的URL
    current_url = "https://gateway.qschou.com/v3.0.0/support/support/{}".format(uuid_cut)
    base_url="https://gateway.qschou.com/v3.0.0/support/support/{}".format(uuid_cut)
#     捐款人数
    for _ in range(int(backer_count)//10):
        response = requests.get(current_url)
        json_data = response.json()
#             获得评论数
        comment_number = len([item['message'] for item in json_data['data'] if len(item['message'])>0 ])
        allpls.append(comment_number)
#             获得回复数
        hf_number = len([item['comments'] for item in json_data['data']])
        allhdpls.append(hf_number)
#             回复字数
        hf_number = sum([len(comment['content']) for item in json_data['data'] for comment in item['comments']])
        allzs.append(hf_number)
        # 获取下一个页面的URL
        next_url=json_data['next']
        next_page_url = base_url+"?next="+next_url
        # 更新当前页面的URL为下一个页面的URL
        current_url = next_page_url
    return sum(allpls),sum(allhdpls),sum(allzs)

In [41]:
# 项目基本信息
alldata=pd.DataFrame()
for i in range(len(uuid_cut)):
    url = 'https://gateway.qschou.com/v3.0.0/project/index/text3/{}?with=detail'.format(uuid_cut[i])
    headers = {'Host': 'gateway.qschou.com', 'Connection': 'keep-alive', 'Content-Length': '9357', 'cookie': '', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 MicroMessenger/7.0.20.1781(0x6700143B) NetType/WIFI MiniProgramEnv/Windows WindowsWechat/WMPF XWEB/6939', 'charset': 'utf-8', 'Accept-Encoding': 'gzip,compress,br,deflate', 'Content-Type': 'application/json; charset=UTF-8','Referer': 'referer: https://servicewechat.com/wx41b2d3a971bf407b/121/page-frame.html'}
    response = requests.get(url)
    data = response.json()
#     项目id
    prouuid=uuid_cut[i]
#     项目标题
    title = data['data']['project']['name']
#     筹资规模
    total_amount = data['data']['project']['target_amount']
#     已筹金额
    current_amount = data['data']['project']['raised_amount']
#     捐款人数
    backer_count = data['data']['project']['support_number']
#     转发次数
    share_count = data['data']['project']['share_number']
#     融资效率
#     正文字数
    detail=data['data']['project']['detail']
    word_count=len(detail)
#     图片数量
    pic= len(data['data']['project']['cover'])
#     创建时间
    created_time=data['data']['project']['created']
    creat_time = datetime.datetime.fromtimestamp(int(created_time))
#     剩余天数
    left_time=data['data']['project']['time_left']
#    地区、年纪
    province,age=get_baseinfo(uuid_cut[i])
#     更新次数、更新图片数、更新字数
    up_times,image_count,content_count=get_jz(uuid_cut[i])
#     亲友认证
    prove_total=get_qy(uuid_cut[i])
#     评论数、回复评论数、回复字数
    allpls, allhdpls,allzs=get_pl(backer_count,uuid_cut[i])
    data=pd.DataFrame({"项目id":[prouuid],'项目标题': title,'地区':province,'病人年纪':age,'筹资规模': total_amount,'已筹金额': current_amount,"捐款人数": backer_count,'转发次数': share_count,'正文字数': word_count,'图片数量': pic,"创建时间":creat_time,"剩余天数":left_time,'项目进展更新次数':up_times,'更新字数':content_count,'更新图片数量':image_count,'亲友认证数量':prove_total,"留言次数":allpls,"回复次数":allhdpls,"回复字数":allzs})
    alldata=pd.concat([alldata,data],ignore_index=True)

In [42]:
alldata

,项目id,项目标题,地区,病人年纪,筹资规模,已筹金额,捐款人数,转发次数,正文字数,图片数量,创建时间,剩余天数,项目进展更新次数,更新字数,更新图片数量,亲友认证数量,留言次数,回复次数,回复字数
0,df34a086-0bf9-40b9-bd29-600d8ace7115,丈夫张金池发生意外车祸，全身受伤严重急需治疗，恳求大家帮助！,山东,33,200000,55519,488,243,726,6,2023-05-25 10:31:37,剩余27天,4,48,7,27,480,480,18180
1,14528720-61be-4a96-9f92-55f965fc04f6,母亲冯月英急性心脏大动脉撕裂出血！生命垂危，恳请各位爱心人士救救她！,,54,300000,22426,737,250,846,3,2023-05-25 10:36:40,剩余27天,5,69,7,14,730,730,24982
2,0be06910-d7ab-45c0-94ec-6d594820b74b,重症ICU门外生命呼救，请大家救救我父亲,湖南,61,250000,8663,185,113,984,8,2023-05-25 11:31:06,剩余27天,2,19,2,8,180,180,6129
3,968beb4b-7eec-4c0e-9554-a0a3bbe6c1f4,丈夫邓勇强你一定要撑下去！两个孩子还在读书，需要爸爸！！！,广东,50,300000,22386,742,434,729,8,2023-05-25 11:34:00,剩余27天,4,33,3,44,740,740,25009
4,26cc4cf5-f16e-437f-b974-91a2c59c6527,父亲黄位宁辛苦了半辈，不幸身患多发性骨髓瘤，急需要移植手术延续生命！,广西,58,490000,17175,648,405,1127,7,2023-05-25 12:06:37,剩余27天,2,19,2,17,640,640,21830
5,61cea2f6-830c-4b42-a71e-7e7a8a48c3db,老婆(保道争）28种重症ICU抢救，生死未卜，恳求大家救救我的老婆,,38,200000,29145,985,991,1019,8,2023-05-25 12:06:59,剩余27天,2,19,2,21,980,980,33242
6,e3ece50a-a6ce-479c-9c0b-33e742454959,恳求好心人救救我的丈夫王荣铭，谢谢你们了！,,47,150000,7148,130,164,639,3,2023-05-25 12:14:42,剩余27天,2,19,2,5,130,130,4448
7,1660de23-17f5-4c5e-b540-90adeb47aefb,紧急求助！丈夫宁颜患脑出血、尿毒症急需治疗，家庭无法负担，请大家帮帮我们！,,42,300000,12633,524,224,690,5,2023-05-25 12:17:53,剩余27天,3,39,1,14,520,520,17806
8,6f0579aa-c097-4b1d-9292-d003c85fd302,急急急！救救孩子！他才10个月，摔伤导致脑出血 脑积液 脑萎缩，急需治疗，恳请大家伸出援手救救他！,云南,0,200000,16140,561,365,777,8,2023-05-25 12:19:26,剩余27天,3,50,2,27,560,560,19146
9,a47592a3-380b-419b-be2c-52de93103a53,生命接力！希望大家可以帮助我活下去，恳请大家伸出援手！！,江西,34,200000,9546,310,173,511,5,2023-05-25 12:39:20,剩余27天,2,19,2,8,310,310,10484


In [38]:
alldata['回复率']=alldata['回复次数']/alldata['留言次数']

In [43]:
alldata.iloc[:, 1:] 

,项目标题,地区,病人年纪,筹资规模,已筹金额,捐款人数,转发次数,正文字数,图片数量,创建时间,剩余天数,项目进展更新次数,更新字数,更新图片数量,亲友认证数量,留言次数,回复次数,回复字数
0,丈夫张金池发生意外车祸，全身受伤严重急需治疗，恳求大家帮助！,山东,33,200000,55519,488,243,726,6,2023-05-25 10:31:37,剩余27天,4,48,7,27,480,480,18180
1,母亲冯月英急性心脏大动脉撕裂出血！生命垂危，恳请各位爱心人士救救她！,,54,300000,22426,737,250,846,3,2023-05-25 10:36:40,剩余27天,5,69,7,14,730,730,24982
2,重症ICU门外生命呼救，请大家救救我父亲,湖南,61,250000,8663,185,113,984,8,2023-05-25 11:31:06,剩余27天,2,19,2,8,180,180,6129
3,丈夫邓勇强你一定要撑下去！两个孩子还在读书，需要爸爸！！！,广东,50,300000,22386,742,434,729,8,2023-05-25 11:34:00,剩余27天,4,33,3,44,740,740,25009
4,父亲黄位宁辛苦了半辈，不幸身患多发性骨髓瘤，急需要移植手术延续生命！,广西,58,490000,17175,648,405,1127,7,2023-05-25 12:06:37,剩余27天,2,19,2,17,640,640,21830
5,老婆(保道争）28种重症ICU抢救，生死未卜，恳求大家救救我的老婆,,38,200000,29145,985,991,1019,8,2023-05-25 12:06:59,剩余27天,2,19,2,21,980,980,33242
6,恳求好心人救救我的丈夫王荣铭，谢谢你们了！,,47,150000,7148,130,164,639,3,2023-05-25 12:14:42,剩余27天,2,19,2,5,130,130,4448
7,紧急求助！丈夫宁颜患脑出血、尿毒症急需治疗，家庭无法负担，请大家帮帮我们！,,42,300000,12633,524,224,690,5,2023-05-25 12:17:53,剩余27天,3,39,1,14,520,520,17806
8,急急急！救救孩子！他才10个月，摔伤导致脑出血 脑积液 脑萎缩，急需治疗，恳请大家伸出援手救救他！,云南,0,200000,16140,561,365,777,8,2023-05-25 12:19:26,剩余27天,3,50,2,27,560,560,19146
9,生命接力！希望大家可以帮助我活下去，恳请大家伸出援手！！,江西,34,200000,9546,310,173,511,5,2023-05-25 12:39:20,剩余27天,2,19,2,8,310,310,10484


In [44]:
alldata.to_csv('轻松筹数据5.28.csv',encoding='utf_8_sig',mode='a',header=True,index=None)